In [3]:
from datasets import load_from_disk
import datasets
dataset = load_from_disk("./resources/data/train_dataset")
val_dataset = dataset['validation']

In [7]:
isinstance(val_dataset, datasets.Dataset)

True

In [9]:
val_dataset.format

{'type': None,
 'format_kwargs': {},
 'columns': ['title',
  'context',
  'question',
  'id',
  'answers',
  'document_id',
  '__index_level_0__'],
 'output_all_columns': False}

In [11]:
val_dataset.set_format(
                type=val_dataset.format["type"],
                columns=list(val_dataset.features.keys()),
            )
val_dataset.format

{'type': None,
 'format_kwargs': {},
 'columns': ['title',
  'context',
  'question',
  'id',
  'answers',
  'document_id',
  '__index_level_0__'],
 'output_all_columns': False}

In [12]:
val_dataset[0]

{'title': '전효숙',
 'context': '순천여자고등학교 졸업, 1973년 이화여자대학교를 졸업하고 1975년 제17회 사법시험에 합격하여 판사로 임용되었고 대법원 재판연구관, 수원지법 부장판사, 사법연수원 교수, 특허법원 부장판사 등을 거쳐 능력을 인정받았다. 2003년 최종영 대법원장의 지명으로 헌법재판소 재판관을 역임하였다.\\n\\n경제민주화위원회(위원장 장하성이 소액주주들을 대표해 한보철강 부실대출에 책임이 있는 이철수 전 제일은행장 등 임원 4명을 상대로 제기한 손해배상청구소송에서 서울지방법원 민사합의17부는 1998년 7월 24일에 "한보철강에 부실 대출하여 은행에 막대한 손해를 끼친 점이 인정된다"며 "원고가 배상을 청구한 400억원 전액을 은행에 배상하라"고 하면서 부실 경영인에 대한 최초의 배상 판결을 했다. \\n\\n2004년 10월 신행정수도의건설을위한특별조치법 위헌 확인 소송에서 9인의 재판관 중 유일하게 각하 견해를 내었다. 소수의견에서 전효숙 재판관은 다수견해의 문제점을 지적하면서 관습헌법 법리를 부정하였다. 전효숙 재판관은 서울대학교 근대법학교육 백주년 기념관에서 열린 강연에서, 국회가 고도의 정치적인 사안을 정치로 풀기보다는 헌법재판소에 무조건 맡겨서 해결하려는 자세는 헌법재판소에게 부담스럽다며 소회를 밝힌 바 있다.',
 'question': '처음으로 부실 경영인에 대한 보상 선고를 받은 회사는?',
 'id': 'mrc-0-003264',
 'answers': {'answer_start': [284], 'text': ['한보철강']},
 'document_id': 9027,
 '__index_level_0__': 2146}

In [24]:
from transformers import HfArgumentParser, AutoTokenizer, AutoModelForCausalLM, DataCollatorForLanguageModeling, BitsAndBytesConfig
from peft import get_peft_model, LoraConfig
import torch

tokenizer = AutoTokenizer.from_pretrained('Qwen/Qwen2.5-1.5B-Instruct')
quantization_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_use_double_quant=True,
        bnb_4bit_compute_dtype=torch.bfloat16
        # bnb_4bit_compute_dtype=torch.float16
)
model = AutoModelForCausalLM.from_pretrained(
    'Qwen/Qwen2.5-1.5B-Instruct',
    from_tf=bool(".ckpt" in 'Qwen/Qwen2.5-1.5B-Instruct'),
    quantization_config=quantization_config
)
lora_config = LoraConfig(r=8, lora_alpha=16, lora_dropout=0.1, bias="none", task_type="CAUSAL_LM", modules_to_save=None,)
model = get_peft_model(model, lora_config)

`low_cpu_mem_usage` was None, now set to True since model is quantized.


In [25]:
from utils.data_processing import GenerationDataModule
dm = GenerationDataModule(data_args, training_args, tokenizer) 
train_dataset, eval_dataset = dm.get_processing_data()
train_dataset, eval_dataset

(Dataset({
     features: ['prompt'],
     num_rows: 3952
 }),
 Dataset({
     features: ['prompt'],
     num_rows: 240
 }))

In [26]:
from model.qat_custom import GenerationTrainer
from utils.arguments_gen_reader import ModelArguments, OurTrainingArguments, DataTrainingArguments
from utils.data_processing import GenerationDataModule
from evaluate import load

parser = HfArgumentParser(
        (ModelArguments, DataTrainingArguments, OurTrainingArguments) # arguement 쭉 읽어보면서 이해하기
    )
model_args, data_args, training_args = parser.parse_args_into_dataclasses()
dm = GenerationDataModule(data_args, training_args, tokenizer) 
train_dataset, eval_dataset = dm.get_processing_data()
metric = load('squad')
trainer = GenerationTrainer(
        model=model,
        dataset_text_field="prompt",
        max_seq_length=2000,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        peft_config=lora_config,
        tokenizer=tokenizer,
        packing= training_args.packing,
        data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False),
        metrics=metric)

usage: ipykernel_launcher.py [-h] [--model_name_or_path MODEL_NAME_OR_PATH]
                             [--dataset_name DATASET_NAME]
                             [--overwrite_cache [OVERWRITE_CACHE]]
                             [--preprocessing_num_workers PREPROCESSING_NUM_WORKERS]
                             [--max_answer_length MAX_ANSWER_LENGTH]
                             [--output_dir OUTPUT_DIR]
                             [--overwrite_output_dir [OVERWRITE_OUTPUT_DIR]]
                             [--do_train [DO_TRAIN]] [--no_do_train]
                             [--do_eval [DO_EVAL]] [--no_do_eval]
                             [--do_predict [DO_PREDICT]]
                             [--eval_strategy {no,steps,epoch}]
                             [--prediction_loss_only [PREDICTION_LOSS_ONLY]]
                             [--per_device_train_batch_size PER_DEVICE_TRAIN_BATCH_SIZE]
                             [--per_device_eval_batch_size PER_DEVICE_EVAL_BATCH_SIZE]
   

SystemExit: 2

/data/ephemeral/home/level2-mrc-nlp-08/.venv/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
trainer.evaluate(val_dataset)

In [8]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel, PeftConfig
import torch

# 기본 모델 경로 (원본 모델의 경로여야 합니다)
base_model_path = "beomi/Qwen2.5-7B-Instruct-kowiki-qa-context"

# Checkpoint 경로
checkpoint_path = "./resources/checkpoint/generation/checkpoint-1482"

# PEFT 설정 로드
peft_config = PeftConfig.from_pretrained(checkpoint_path)

# 기본 모델 경로
base_model_path = peft_config.base_model_name_or_path

# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(base_model_path, trust_remote_code=True)

# 양자화 설정
quantization_config = BitsAndBytesConfig(load_in_8bit=True)

# 기본 모델 로드
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_path,
    quantization_config=quantization_config,
    device_map="auto",
    trust_remote_code=True
    
)


# PEFT 모델 로드
model = PeftModel.from_pretrained(base_model, checkpoint_path)

# 모델 평가 모드로 설정

def generate_prompt(question, context):
    # Qwen prefix
    prefix_chat_template = '''<|im_start|>system
    You are Qwen, created by Alibaba Cloud. You are a helpful assistant. 모든 대답은 한국어로 해주세요.<|im_end|>
    <|im_start|>user
    question:{} 
    context:{}<|im_end|>
    <|im_start|>assistant
    <|im_end|>'''

    # prefix에 formatting
    prompts = [prefix_chat_template.format(question, context)]
    # 데이터에 prompt 추가
    return prompts

# 추론 함수
def inference(question, context):
    model.eval()
    inputs = tokenizer(generate_prompt(question, context), return_tensors="pt", padding=True, truncation=True, max_length=2000)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    print(inputs)
    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=50)
    
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [9]:
context = ''''ㅋㅋ'은 일반적으로 웃음소리를 표현할 때 사용된다. 통신체에서는 웃는 모습을 직접적으로 보여줄 수 없기 때문에 의성어를 사용하는 경우가 많은데 그 중에서 '킥킥', '큭큭'을 초성체로 바꾸어 사용한 것이 'ㅋㅋ'에 해당한다. 이에 대해 2001년 연구에서는 "자음만 가지고 표시하여 통신상의 재미를 더하기 위한 방법으로 사용되고 있다"고 해석한 바 있다. \n\nㅋ자를 웃음소리를 의미하는 것으로 사용할 때, 'ㅋ'자 한 개에서 'ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ...'하는 식으로 열 타 이상 한 줄 넘게까지 쓰이기도 한다 ㅋ자를 얼마나 많이 썼느냐에 따라 그 느낌과 의미는 달라진다. 신조어를 전문으로 연구하는 이재현 문화평론가는 《한국일보》의 기고글에서 그 종류와 느낌이 대략 다음과 같다고 보았다\n* ㅋ - 일반적으로 무심하게 동의하려는 상황에서 쓰인다. 다른 말끝에 붙였을 때, 이를테면 '그래ㅋ', '좋네ㅋ', '올ㅋ' 등은 웃음 자체라기보다는 입꼬리가 아주 살짝 올라간 상황이라는 느낌을 준다. 의성어로는 "큭" 혹은 "킥"으로 해석될 수 있다. 한편 '뭐해?ㅋ', '미안ㅋ'의 경우 'ㅋ'는 상투적인 군말의 역할을 한다.\n* ㅋㅋ - 위의 것보다 ㅋ자가 하나가 더 많으나 오히려 형식적이라는 느낌을 준다. 좋게는 '그렇군'을 뜻하거나, 조금 나쁘게는 대화상에서 의례적으로 추임새를 넣고 있다는 느낌을 준다. 'ㅋㅋ'은 미혼남녀들이 가장 싫어하는 성의 없는 메신저 말투로 선정되기도 했다. 다만 회사 상사에게 대답하는 경우 같은 공적인 상황에서는 '네ㅋㅋ'와 같은 대답은 가벼워 보인다는 조사도 있었다. \n* ㅋㅋㅋ - 비교적 중립적이면서도 가장 많이 사용되는 용법이다. 웃는 감정을 그대로 표현한다는 뉘앙스를 갖는다.\n* ㅋㅋㅋㅋ... - 여기서부터는 "정말 웃긴다"라는 반응을 나름 정성들여서 표현하는 느낌을 준다. 네 개부터 그 이상은 의미의 함축과 정서의 강도가 거의 같으며, 이런 점에서 'ㅋㅋㅋㅋ'는 실질적으로 웃긴다는 것을 표현하는 최소 조건에 해당한다.'''
question = '웃는 감정을 그대로 표현한다는 뉘앙스를 갖는 ㅋ의 용법은?'
generated_text = inference(context, question)
print(f"Generated text: {generated_text}")

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


{'input_ids': tensor([[151644,   8948,    198,    262,   1446,    525,   1207,  16948,     11,
           3465,    553,  54364,  14817,     13,   1446,    525,    264,  10950,
          17847,     13, 129304,  60960, 132760,  33704, 130092,  31079,  17380,
          60716,  91669,     13, 151645,    198,    257, 151644,    872,    198,
            262,   3405,   4884, 144410, 144410,      6,  33704, 134664, 128552,
          85413,    225,  48431,  43590, 133886, 139053,  47836,  53618,  40720,
         132306,     13, 125206,  82528,  49543, 129889,  85413,    225,  16560,
         136665,  17877, 137352, 128552,  63332,  57026, 131303,  28733,  46682,
          20487, 129147, 124970,  32831,  31079,  18411,  40720,  42905,  49052,
          19969, 129875, 124419,  54825,  70943,  56475,    364, 145329, 145329,
            516,    364, 147758, 147758,      6,  17877,  83315,  32831,  49543,
          17380,  81718, 136892,  31079,  40720,  23573, 128584,    364, 144410,
         14441